In [1]:
import aoc

data = aoc.read("day19.txt")
workflows_str = data[0].splitlines()
ratings_str = data[1].splitlines()

In [2]:
ratings = []
for line in ratings_str:
    values = {}
    line = line[1:-1].split(",")
    values = {r[0]: int(r[2:]) for r in line}
    ratings.append(values)

In [3]:
workflows = {}


def parse_workflow_line(line):
    first_accolade = line.index("{")
    name = line[:first_accolade]
    instructions = line[first_accolade + 1 : -1].split(",")
    return name, instructions


def parse_comparison(comparison):
    if ":" not in comparison:
        return comparison
    key = comparison[0]
    comparator = comparison[1]
    ix_colon = comparison.index(":")
    value = int(comparison[2:ix_colon])
    outcome_if_true = comparison[ix_colon + 1 :]
    return key, comparator, value, outcome_if_true


for line in workflows_str:
    name, instructions = parse_workflow_line(line)
    workflows[name] = [parse_comparison(comparison) for comparison in instructions]

# Part 1

In [ ]:
def handle_comparison(rating, comparison):
    if isinstance(comparison, str):
        return comparison  # The final "else"-clause
    key, comparator, value, outcome_if_true = comparison
    first_value = rating[key]
    if comparator == ">":
        comparison_outc = first_value > value
    elif comparator == "<":
        comparison_outc = first_value < value
    else:
        assert False

    if comparison_outc:
        return outcome_if_true
    return False


def handle_workflow(rating, workflow):
    know_what_to_do = False
    i = 0
    while not know_what_to_do:
        comparison = workflow[i]
        know_what_to_do = handle_comparison(rating, comparison)
        i += 1
    return know_what_to_do


def handle_rating(rating):
    current_workflow_name = "in"
    while current_workflow_name not in "AR":
        current_workflow = workflows[current_workflow_name]
        current_workflow_name = handle_workflow(rating, current_workflow)
    return current_workflow_name


accepted_ratings = []
for rating in ratings:
    outc = handle_rating(rating)
    if outc == "A":
        accepted_ratings.append(rating)
sum(sum(d.values()) for d in accepted_ratings)

# Part 2

In [5]:
from collections import deque

q = deque([("in", {"x": (1, 4000), "m": (1, 4000), "a": (1, 4000), "s": (1, 4000)})])


def handle_workflow_with_options(options, workflow):
    new_options = []
    for comparison in workflow:
        if isinstance(comparison, str):  # The final "else"-clause
            new_options.append((comparison, options))
            continue
        key, comparator, value, outcome_if_true = comparison
        true_options = options.copy()
        remaining_options = options.copy()
        min_, max_ = true_options[key]
        if comparator == "<":
            new_max = min(max_, value - 1)
            true_options[key] = (min_, new_max)
            remaining_options[key] = (new_max + 1, max_)

            new_options.append((outcome_if_true, true_options))

            if new_max == max_:
                break
        elif comparator == ">":
            new_min = max(min_, value + 1)
            true_options[key] = (new_min, max_)
            remaining_options[key] = (min_, new_min - 1)
            new_options.append((outcome_if_true, true_options))
            if new_min == min_:
                break
        options = remaining_options.copy()
    return new_options


accepted_ratings = []
while q:
    current_workflow_name, options = q.popleft()
    if current_workflow_name == "A":
        accepted_ratings.append(options)
        continue
    if current_workflow_name == "R":
        continue
    workflow = workflows[current_workflow_name]
    new_options = handle_workflow_with_options(options, workflow)
    q.extend(new_options)

In [ ]:
def calc_n_options(options):
    n = 1
    for min_, max_ in options.values():
        n *= max_ + 1 - min_
    return n


sum(calc_n_options(option) for option in accepted_ratings)